In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("movieList2.csv")
df.head()


,Project Title,Project Title (Original Language),Poster,First Name,Last Name,Full Name,Email,Phone,Address,Address 2,...,Rating,Recommendations,Recommendations: Pass,Recommendations: Recommend,Recommendations: Award Worthy,Recommendations: Maybe,Flag,Submission Date,Submission Categories,Screenings / Awards
0,SCARS,NaN,NaN,Ujjwal Raj,Rai,Ujjwal Raj,rai_ujjwal@yahoo.com,0420417897,Unit 1610/120 Eastern Valley Way,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-09,International Short Film,"Canberra\nMarch 8, 2024\nLights Canberra Actio..."
1,The Cut,NaN,NaN,Ian Liu,Liu,Ian Liu,ianliu857@gmail.cm,414626221,33 Rose Lane,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-13,International Short Film,NaN
2,Al Bangeri,البنجري,NaN,Moosa Alkindi,Alkindi,Moosa Alkindi,moosa@royalpro.net,+968 95120339,Oman-Muscat/Alkhoud 7,Royal Agency,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-14,International Short Film,"Qade Film Festival\nMuscat\nFebruary 3, 2024\n..."
3,Lilith,NaN,NaN,Dale Crawford,Crawford,Dale Crawford,info@dalecrawford.org,+61400217223,55 McSwain St,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-14,International Feature Film,NaN
4,The Night's Takings,NaN,NaN,Ahmed Al Aasi,Al Aasi,Ahmed Al Aasi,aelassy@gmail.com,+61435838429,Unit 109 120 Palmer Street,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-17,International Short Film,NaN


In [5]:
df.info

<bound method DataFrame.info of                                     Project Title  \
0                                           SCARS   
1                                         The Cut   
2                                      Al Bangeri   
3                                          Lilith   
4                             The Night's Takings   
..                                            ...   
105                                       In Half   
106  Chernobyl - A Bomb That Keeps Ticking (2024)   
107                                          ERAN   
108                                  Lasya's Song   
109                                          BAAZ   

    Project Title (Original Language)  Poster      First Name    Last Name  \
0                                 NaN      NaN     Ujjwal Raj          Rai   
1                                 NaN      NaN        Ian Liu          Liu   
2                             البنجري      NaN  Moosa Alkindi      Alkindi   
3                  

In [6]:
df.isnull().sum()

Project Title                          0
Project Title (Original Language)     87
Poster                               110
First Name                             0
Last Name                              1
Full Name                              0
Email                                  0
Phone                                 34
Address                               40
Address 2                             92
City                                  34
State                                 38
Postal Code                           39
Full Address                           0
Country                               35
Tracking Number                        0
Synopsis                               3
Synopsis (Original Language)          93
Lyrics                               110
Duration                               0
Country of Origin                      1
Language                               7
Trailer URL                           41
Country of Filming                     4
Project Website 

In [7]:
df.columns

Index(['Project Title', 'Project Title (Original Language)', 'Poster ',
       'First Name', 'Last Name', 'Full Name', 'Email', 'Phone', 'Address',
       'Address 2', 'City', 'State', 'Postal Code', 'Full Address', 'Country',
       'Tracking Number', 'Synopsis', 'Synopsis (Original Language)', 'Lyrics',
       'Duration', 'Country of Origin', 'Language', 'Trailer URL',
       'Country of Filming', 'Project Website', 'Twitter', 'Facebook',
       'Instagram', 'Project Type', 'Genres', 'Student Project',
       'Completion Date', 'Production Budget', 'Shooting Format',
       'Aspect Ratio', 'Film Color', 'Focal Length', 'Shutter Speed',
       'Aperture', 'ISO / Film', 'First-time Filmmaker', 'Directors',
       'Writers', 'Producers', 'Key Cast', 'Other Credits',
       'Submitter Statement', 'Submitter Biography', 'Rating',
       'Recommendations', 'Recommendations: Pass',
       'Recommendations: Recommend', 'Recommendations: Award Worthy',
       'Recommendations: Maybe', 'Flag',

In [8]:
df = df[['Project Title', 'Poster ', 'Country',
       'Tracking Number', 'Synopsis',
       'Duration', 'Country of Origin', 'Language',
       'Country of Filming', 'Genres', 'Student Project', 'Shooting Format',
       'Aspect Ratio', 'Film Color', 'Directors',
       'Writers', 'Producers', 'Key Cast', 'Rating',
       'Recommendations',
       'Submission Categories', 'Screenings / Awards']]

In [9]:
df.head()

,Project Title,Poster,Country,Tracking Number,Synopsis,Duration,Country of Origin,Language,Country of Filming,Genres,...,Aspect Ratio,Film Color,Directors,Writers,Producers,Key Cast,Rating,Recommendations,Submission Categories,Screenings / Awards
0,SCARS,NaN,Australia,IFFA1001,A tennis sensation struggles to deal with the ...,00:07:46,Australia,English,Australia,NaN,...,16:9,Color,Ujjwal Rai,Ujjwal Rai,Ujjwal Rai,"Lily Welling, Kristofer Patson Gill, Charis Ba...",NaN,NaN,International Short Film,"Canberra\nMarch 8, 2024\nLights Canberra Actio..."
1,The Cut,NaN,Australia,IFFA1002,A 30 years old actress Kate has stayed unemplo...,00:12:00,Australia,English,Australia,NaN,...,4:3,Black & White and Color,Xunyan Liu,Xunyan Liu,Xunyan Liu,"Azaleia Bradley, Tommy Corrigan, Serkan Tokgoz",NaN,NaN,International Short Film,NaN
2,Al Bangeri,NaN,Oman,IFFA1003,"Haseena, the protagonist of our film, a widow ...",00:25:11,Oman,Arabic,Oman,"Drama, Short Film",...,16:9,Color,Moosa Nasser Alkindi,Haleema Ahmed Alka'abi,Muneer Khalfan Al Rawahi,NaN,NaN,NaN,International Short Film,"Qade Film Festival\nMuscat\nFebruary 3, 2024\n..."
3,Lilith,NaN,Australia,IFFA1004,Fearing for his family’s safety after the murd...,01:32:35,Australia,English,Australia,NaN,...,2:1,Color,Dale Edward Crawford,Dale Edward Crawford,Dale Edward Crawford,"William Emmons, Marie-Therese Bjornerud, Lila ...",NaN,NaN,International Feature Film,NaN
4,The Night's Takings,NaN,Australia,IFFA1005,A man steals the night's takings from the bar ...,00:08:14,Australia,English,Australia,NaN,...,NaN,Color,Ahmed Al Aasi,Colin J Harte,Ahmed Al Aasi,"Joanna Bakker, Colin J Harte, John Robert Spar...",NaN,NaN,International Short Film,NaN


In [10]:
df.columns

Index(['Project Title', 'Poster ', 'Country', 'Tracking Number', 'Synopsis',
       'Duration', 'Country of Origin', 'Language', 'Country of Filming',
       'Genres', 'Student Project', 'Shooting Format', 'Aspect Ratio',
       'Film Color', 'Directors', 'Writers', 'Producers', 'Key Cast', 'Rating',
       'Recommendations', 'Submission Categories', 'Screenings / Awards'],
      dtype='object')

In [11]:
df.isnull().sum()

Project Title              0
Poster                   110
Country                   35
Tracking Number            0
Synopsis                   3
Duration                   0
Country of Origin          1
Language                   7
Country of Filming         4
Genres                    59
Student Project            0
Shooting Format           13
Aspect Ratio              32
Film Color                 0
Directors                  1
Writers                   12
Producers                  5
Key Cast                  19
Rating                   110
Recommendations          110
Submission Categories      0
Screenings / Awards       83
dtype: int64

In [12]:
df['Aspect Ratio'].dropna()

0      16:9
1       4:3
2      16:9
3       2:1
5      16:9
       ... 
104    16:9
106    16:9
107    16:9
108    16:9
109    16:9
Name: Aspect Ratio, Length: 78, dtype: object

In [13]:
df['Aspect Ratio'].isnull().sum()

32

In [14]:
df['Aspect Ratio']=df['Aspect Ratio'].isnull().sum()

C:\Users\Harsh\AppData\Local\Temp\ipykernel_24924\3542903292.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Aspect Ratio']=df['Aspect Ratio'].isnull().sum()


In [15]:
df.isnull().sum()

Project Title              0
Poster                   110
Country                   35
Tracking Number            0
Synopsis                   3
Duration                   0
Country of Origin          1
Language                   7
Country of Filming         4
Genres                    59
Student Project            0
Shooting Format           13
Aspect Ratio               0
Film Color                 0
Directors                  1
Writers                   12
Producers                  5
Key Cast                  19
Rating                   110
Recommendations          110
Submission Categories      0
Screenings / Awards       83
dtype: int64

In [16]:
''' Missing value columns - 
1. Posters 
2. COuntry 
3. Genres
4. Shooting format
5. Writers
6. Key cast 
7. Ratings 
8. Recommendations 
9. Screenings / Awards 
'''

' Missing value columns - \n1. Posters \n2. COuntry \n3. Genres\n4. Shooting format\n5. Writers\n6. Key cast \n7. Ratings \n8. Recommendations \n9. Screenings / Awards \n'

In [17]:
df['Project Title']

0                                             SCARS
1                                           The Cut
2                                        Al Bangeri
3                                            Lilith
4                               The Night's Takings
                           ...                     
105                                         In Half
106    Chernobyl - A Bomb That Keeps Ticking (2024)
107                                            ERAN
108                                    Lasya's Song
109                                            BAAZ
Name: Project Title, Length: 110, dtype: object

In [18]:
df.columns

Index(['Project Title', 'Poster ', 'Country', 'Tracking Number', 'Synopsis',
       'Duration', 'Country of Origin', 'Language', 'Country of Filming',
       'Genres', 'Student Project', 'Shooting Format', 'Aspect Ratio',
       'Film Color', 'Directors', 'Writers', 'Producers', 'Key Cast', 'Rating',
       'Recommendations', 'Submission Categories', 'Screenings / Awards'],
      dtype='object')

In [34]:
df.isnull().sum()

Project Title              0
Poster                   110
Country                   35
Tracking Number            0
Synopsis                   3
Duration                   0
Country of Origin          1
Language                   7
Country of Filming         4
Genres                    59
Student Project            0
Shooting Format           13
Aspect Ratio               0
Film Color                 0
Directors                  1
Writers                   12
Producers                  5
Key Cast                  19
Rating                   110
Recommendations          110
Submission Categories      0
Screenings / Awards       83
dtype: int64

In [36]:
df.to_csv('Movie_list.csv')